In [ ]:
import ipywidgets as widgets

from google.cloud import storage
from datetime import datetime
from tqdm.notebook import tqdm
from IPython.display import display

# download the selected sample files
import tempfile

from fastavro import reader
import numpy as np
import pandas as pd
from glob import glob

from datetime import datetime

In [ ]:
class Storage:
    def __init__(self):
        self.client = storage.Client(project='augury-ctoo')
        self.refresh_blobs()
        self.timestamps = []
        
    def refresh_blobs(self):
        self.blobs = []
        for blob in self.client.list_blobs("transformers-ae"):
            self.blobs.append(blob)

    def get_ids(self):
        ids = []
        for blob in self.blobs:
            id = blob.name.split("_")[0] + "_" + blob.name.split("_")[1]
            if id not in ids:
                ids.append(id)

        return ids
    
    def get_timestamps(self, id):
        timestamps = []
        for blob in self.blobs:
            if blob.name.split("_")[0] + "_" + blob.name.split("_")[1] != id:
                continue

            ts = int(blob.name.split("_")[2].split('.')[0])
            o = (datetime.utcfromtimestamp(ts), blob)
            if o not in timestamps:
                timestamps.append(o)
                
        return timestamps
    
def search(id):
    s.timestamps = s.get_timestamps(id)
    if len(s.timestamps) == 0:
        box2.layout = {'visibility': 'hidden'}
        return

    options = []
    for t in s.timestamps:
        options.append(t[0])
        
    selection_range_slider.options = options
    selection_range_slider.index = (0, len(options)-1)
    box2.layout = {'visibility': 'visible'}

def on_dropdown_change(d):
    search(d.new)
    
def on_go_button_clicked(h):
    global blobs
    blobs = []
    for t in s.timestamps[selection_range_slider.index[0]:selection_range_slider.index[1]+1]:
        blobs.append((t[0], t[1]))

In [ ]:
s = Storage()

timestamps = [0]
selection_range_slider = widgets.SelectionRangeSlider(
    options=timestamps,
    index=(0, len(timestamps)-1),
    orientation='vertical',
    layout={'width': '500px'},
)

dropdown = widgets.Dropdown(
    options=s.get_ids(),
    description="id"
)

go_button = widgets.Button(description="go!")
box1 = widgets.HBox([dropdown])
box2 = widgets.VBox([selection_range_slider, go_button], layout={'visibility': 'hidden'})

 
dropdown.observe(on_dropdown_change, 'value')
search(dropdown.value)
(dropdown.options)

blobs = []

go_button.on_click(on_go_button_clicked)

widgets.VBox([box1, box2])

In [ ]:
tempfiles = []
with tqdm(total=len(blobs)) as pbar:
    for (timestamp, blob) in blobs:
            pbar.set_description("Downloading %s" % blob.name)
            fd, path = tempfile.mkstemp()
            blob.download_to_filename(path)
            tempfiles.append((blob.name, path, timestamp)) #, dropdown.value))
            pbar.update(1)

In [ ]:
with tqdm(total=len(tempfiles)) as pbar:
    for (blob, path, timestamp) in tempfiles:
        with open(path, 'rb') as f:
            for d in reader(f):
                pbar.set_description("Exporting %s" % blob)
                periode = 1000 * 1 / d['samplerate']
                duration = np.arange(0, len(d['adc_signal_mv'][0]) * periode, periode)

                df = None
                if len(d['adc_signal_mv']) == 1:
                    df = pd.DataFrame(list(zip(duration, d['adc_signal_mv'][0])), columns=['time_ms', 'adc_signal_mv'])
                elif len(d['adc_signal_mv']) == 4:
                    df = pd.DataFrame(list(
                        zip(duration, 
                            d['adc_signal_mv'][0], 
                            d['adc_signal_mv'][1], 
                            d['adc_signal_mv'][2], 
                            d['adc_signal_mv'][3])), 
                            columns=['time_ms', 'ch0_adc_signal_mv', 'ch1_adc_signal_mv', 'ch2_adc_signal_mv', 'ch3_adc_signal_mv'])

                df.to_feather("data/{}.fthr".format(blob.split(".avro")[0]))
                pbar.update(1)